In [1]:
import os
import math
import torch
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision import datasets, transforms

In [ ]:
data_dir = "/home/pervinco/Datasets/women-clothes"
num_workers = os.cpu_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lr = 0.0001
batch_size = 128
x_dim = 3
z_dim = 256
w_dim = 256

In [ ]:
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(data_dir, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class MappingNetwork(nn.Module):
    def __init__(self, input_dim=512, output_dim=512, num_layers=8, layer_dim=512):
        super().__init__()
        layers = [nn.Sequential(nn.Linear(input_dim, layer_dim), nn.LeakyReLU(0.2))]
        
        for _ in range(num_layers - 2):
            layers.append(nn.Sequential(nn.Linear(layer_dim, layer_dim), nn.LeakyReLU(0.2)))
        
        layers.append(nn.Linear(layer_dim, output_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, z):
        w = self.model(z)
        return w